In [1]:
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 9.0 MB/s eta 0:00:00


In [2]:
!pip install -q fastapi uvicorn nest_asyncio
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [3]:
import nest_asyncio
import uvicorn
import threading
import urllib.request
from datetime import datetime
from typing import Optional
from uuid import uuid4
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

nest_asyncio.apply()

app = FastAPI()

In [5]:
class User(BaseModel):
    model_config = {"extra": "forbid"}
    __users__ = []

    name: str = Field(..., description="Name of the user")
    email: EmailStr = Field(..., description="Email address of the user")
    friends: list[UUID4] = Field(default_factory=list, max_length=500)
    blocked: list[UUID4] = Field(default_factory=list, max_length=500)
    signup_ts: Optional[datetime] = Field(default_factory=datetime.now, kw_only=True)
    id: UUID4 = Field(default_factory=uuid4, kw_only=True)

    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID4) -> str:
        return str(id)

@app.get("/users", response_model=list[User])
async def get_users() -> list[User]:
    return list(User.__users__)

@app.post("/users", response_model=User)
async def create_user(user: User):
    User.__users__.append(user)
    return user

@app.get("/users/{user_id}", response_model=User)
async def get_user(user_id: UUID4) -> User | JSONResponse:
    try:
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"})

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [6]:
server_thread = threading.Thread(target=run_server)
server_thread.start()

external_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
print(f"\n⚠️ IMPORTANTE: A senha do tunnel é: {external_ip}")
print(f"⚠️ Copie o IP acima, clique no link abaixo e cole o IP na página que abrir.\n")

!lt --port 8000

INFO:     Started server process [357]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



⚠️ IMPORTANTE: A senha do tunnel é: 35.188.2.10
⚠️ Copie o IP acima, clique no link abaixo e cole o IP na página que abrir.

your url is: https://quick-jeans-repair.loca.lt
INFO:     179.155.133.85:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     179.155.133.85:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     179.155.133.85:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     179.155.133.85:0 - "GET /openapi.json HTTP/1.1" 200 OK


/usr/local/lib/python3.12/dist-packages/fastapi/openapi/utils.py:245: UserWarning: Duplicate Operation ID get_users_users_get for function get_users
  warnings.warn(message, stacklevel=1)
/usr/local/lib/python3.12/dist-packages/fastapi/openapi/utils.py:245: UserWarning: Duplicate Operation ID create_user_users_post for function create_user
  warnings.warn(message, stacklevel=1)
/usr/local/lib/python3.12/dist-packages/fastapi/openapi/utils.py:245: UserWarning: Duplicate Operation ID get_user_users__user_id__get for function get_user
  warnings.warn(message, stacklevel=1)


INFO:     179.155.133.85:0 - "POST /users HTTP/1.1" 422 Unprocessable Entity
INFO:     179.155.133.85:0 - "POST /users HTTP/1.1" 422 Unprocessable Entity
INFO:     179.155.133.85:0 - "POST /users HTTP/1.1" 200 OK
INFO:     179.155.133.85:0 - "GET /users HTTP/1.1" 200 OK
INFO:     179.155.133.85:0 - "GET /users HTTP/1.1" 200 OK
INFO:     179.155.133.85:0 - "GET /users/67e9f1fe-7f8d-45e0-9239-d46b195c391d HTTP/1.1" 200 OK
INFO:     179.155.133.85:0 - "GET /users/67e9f1fe-7f8d-45e0-9239-d46b195c391d HTTP/1.1" 200 OK
INFO:     179.155.133.85:0 - "GET /users/67e9f1fe-7f8d-45e0-9239-d46b195c391d HTTP/1.1" 200 OK
INFO:     179.155.133.85:0 - "GET /users/67e9f1fe-7f8d-45e0-9239-d46b195c391d HTTP/1.1" 200 OK
INFO:     179.155.133.85:0 - "GET /users/67e9f1fe-7f8d-45e0-9239-d46b195c391d HTTP/1.1" 200 OK
INFO:     179.155.133.85:0 - "GET /users/1 HTTP/1.1" 422 Unprocessable Entity
INFO:     179.155.133.85:0 - "GET /users/67e9f1fe-7f8d-45e0-9239-d46b195c391d HTTP/1.1" 200 OK
INFO:     179.155.133.8